In [17]:
import requests
import pandas as pd
import hmac
import hashlib
from datetime import datetime

In [62]:
import credentials
secret_bytes = bytes(credentials.secret , 'latin-1')

In [155]:
# rates = requests.get("https://api.coinbase.com/v2/exchange-rates").json()["data"]["rates"]

In [60]:
def generate_access_sign(timestamp, method, requestPath, body):
    message = timestamp + method + requestPath + body
    message_bytes = bytes(message , 'latin-1')
    signature = hmac.new(secret_bytes, message_bytes, digestmod = hashlib.sha256).hexdigest()
    return signature

In [100]:
BASE = "https://api.coinbase.com"
PATH = "/v2/accounts?limit=300"
timestamp = str(round(datetime.now().timestamp()))
signature = generate_access_sign(timestamp, "GET", PATH, "")
HEADERS = {
    "CB-ACCESS-KEY": credentials.key,
    "CB-ACCESS-SIGN": signature,
    "CB-ACCESS-TIMESTAMP": timestamp
}
wallets_request = requests.get(BASE+PATH, headers=HEADERS)

In [217]:
assets = pd.DataFrame.from_dict(wallets_request.json()["data"])
assets["balance"] = assets["native_balance"].apply(lambda balance: float(balance["amount"]))
assets = assets[["currency", "balance"]].set_index("currency")
assets.rename_axis("symbol", inplace=True)

In [211]:
mc_dicts = []
for page in range(1, 20):
    results = requests.get(f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=250&page={page}&sparkline=false").json()
    mc_dicts.extend(results)

In [218]:
mc = pd.DataFrame.from_dict(mc_dicts)
mc["symbol"] = mc["symbol"].str.upper()
mc.set_index("symbol", inplace=True)
mc = mc[["market_cap"]]

In [220]:
df = assets.join(mc)

In [214]:
weights = pd.read_csv('active_weights.csv', index_col="symbol")
weights["weight"] = pd.to_numeric(weights["weight"])
df = df.join(weights)

In [223]:
# df[~df["market_cap"].isnull()]